# Example of using the advertrain library

In [8]:
import torch
from torch import nn, optim
from pathlib import Path
import data_all
from robustML.advertrain.models import ConvNet, ResNet, ConvNetDropblock, ResNetDropblock
from robustML.advertrain.training.classical_training import ClassicalTraining
from robustML.advertrain.training.adversarial_training import AdversarialTraining
from robustML.advertrain.training.autoattack_training import AutoAttackTraining
from robustML.advertrain.training.fire_training import FIRETraining
from robustML.advertrain.training.trades_training import TRADESTraining
from robustML.advertrain.transforms import DataTransformations
import sys
import os
import argparse

In [9]:
# Set here the path to the directory containing the folder c00 of the example dataset. it can be downloaded
# here http://minio-storage.apps.confianceai-public.irtsysx.fr/ml-models/robust-ml-dataset.zip

ROOT_DIR = "/home/jovyan/Maturation/as434_robustness_platform/examples/dataset/c00" 

DEVICE = "cuda:1"
ARCHITECTURE = "resnetdropblock"
LEARNING_RATE = 1e-4
MODELS_PATH = "results/"

In [10]:
transformer = DataTransformations()
train_transform = transformer.get_train_transforms()
test_transform = transformer.get_test_transforms()

In [11]:
path_data = "learning_data/"

train_dataloader1, val_dataloader1 = data_all.get_train_dataloader(
    ROOT_DIR,
    path_data+"train_dataset_all.csv",
    path_data+"val_dataset_all.csv",
    batch_size=16,
    train_transform=train_transform,
    test_transform=test_transform,
)

In [12]:
if ARCHITECTURE == "convnet":
    model = ConvNet(DEVICE)
elif ARCHITECTURE == "resnet":
    model = ResNet(DEVICE)
elif ARCHITECTURE == "convnetdropblock":
    model = ConvNetDropblock(DEVICE)
elif ARCHITECTURE == "resnetdropblock":
    model = ResNetDropblock(DEVICE)
else:
    raise ValueError("Unknown architecture")

In [13]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(weight=torch.Tensor([1, 1]).to(DEVICE))

In [14]:
if not torch.cuda.is_available() or DEVICE =="cpu":
        print("ERROR : Learning should be done on a system with GPU")
        sys.exit()
        
print(torch.cuda.current_device())
print("GPU device used ", DEVICE)

0
GPU device used  cuda:1


## Train class

### ClassicalTraining

In [15]:
METHODE="vanilla"

In [16]:
trainer = ClassicalTraining(model, optimizer, criterion, DEVICE)

### AdversarialTraining

In [17]:
METHODE = "adversarial_training"
EPSILON = 8 / 255

In [18]:
trainer = AdversarialTraining(model, optimizer, criterion, DEVICE, EPSILON)

### AutoAttackTraining

In [19]:
METHODE = "autoattack_training"
EPSILON = 8 / 255

In [20]:
trainer = AutoAttackTraining(model, optimizer, criterion, DEVICE, "ce", EPSILON)

### FireTraining

In [21]:
METHODE = "fire_training"
EPSILON = 8 / 255
BETA = 1

In [22]:
trainer = FIRETraining(model, optimizer, DEVICE, EPSILON, BETA)

### TradesTraining

In [23]:
METHODE = "trades_training"
EPSILON = 8 / 255
BETA = 1

In [24]:
trainer = TRADESTraining(model, optimizer, DEVICE, EPSILON, BETA)

## Fit the model

In [25]:
path = os.path.join(MODELS_PATH,
                    f"{ARCHITECTURE}"+"_"+f"{METHODE}")

if not os.path.exists(path):
    try:
        os.makedirs(path)
    except FileExistsError:
        pass

print(f"Using '{path}' directory")

trainer.fit(epochs=2, train_dataloader=train_dataloader1, val_dataloader=val_dataloader1, patience=2, checkpoint=path)

Using 'results/resnetdropblock_trades_training' directory


/home/jovyan/Maturation/env-robustml/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
Validation: 100%|██████████| 33/33 [00:08<00:00,  4.08it/s]


Epoch 1/2
Train Loss:  0.674, Acc:  0.689, Recall:  0.663, Precision:  0.697,F1 Score:  0.680
Validation Loss:  0.586, Acc:  0.782, Recall:  0.784, Precision:  0.750,F1 Score:  0.767
Validation loss decreased (inf --> 0.586476). Saving model ...


Validation: 100%|██████████| 33/33 [00:08<00:00,  4.10it/s]


Epoch 2/2
Train Loss:  0.546, Acc:  0.791, Recall:  0.772, Precision:  0.805,F1 Score:  0.788
Validation Loss:  0.494, Acc:  0.830, Recall:  0.859, Precision:  0.787,F1 Score:  0.821
Validation loss decreased (0.586476 --> 0.494220). Saving model ...


{'loss': [0.6737884283065796, 0.5462934970855713],
 'acc': [0.6889166831970215, 0.7914999723434448],
 'val_loss': [0.5864764451980591, 0.4942202866077423],
 'val_acc': [0.7821969985961914, 0.8295454382896423],
 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: False\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0\n)',
 'loss_func': 'None',
 'epochs': 2,
 'patience': 2}